## Team Caffeine: Audio Fiend Machine

This demonstration is provided 

The purpose of this project is to demonstrate the software offloading capabilities of the ZYNQ SoC on the PYNQ board. This application uses a modified base overlay that configures the microphone to interface indirectly, through a series of filters, to the four user LEDs. It is set up to identify tones at 1, 3, 5, and 7 kHz though these values can be defined in hardware by filter choice. This audio demodulation capability is a proof of concept for remote control (using sound waves) that does not stress the A9 core. 

![](FBD.png)

## Command Format
Commands are sent with a leading byte of data: b'01111110'
Then the next byte is of the format: b'000xxxxx'
Anything following 4 symbols after the start byte is ignored.
    This data format allows up to 32 unique commands and protects against sending the start byte.
Each symbol represents two bits, and is represented by one of a 1, 3, 5, or 7 kHz sine wave lasting 0.25 seconds. 
For the transmission of 16 bits = 2 bytes in 2 seconds this protocol makes 8 bits per second.

## Decoder Implementation
The decoder is designed using System Generator. First the PDM signal output of the microphone is simulated. A low pass filter is applied to the PDM signal to generate an approximate waveform. This waveform is then downsampled and conditioned before the next step. The signal then passes in parallel through four separate bandpass filters corresponding to the frequency in question. This output is used at the AIX interface back to the A9 microcontroller as well as at the four user LEDs. 

The system generated in SG is then placed into a new copy of the base overlay, careful to supply necessary peripherals such as a clock for the microphone. This is ultimately synthesized and stored on the PYNQ as a bitstream. When the base overlay is called it properly sets the FPGA to perform the decoding.

## Controller Code
The controller code was written for PC platforms on Python3. It uses the simpleaudio (https://pypi.python.org/pypi/simpleaudio/1.0.0) library to generate the necessary tones.

The controller code is NOT expected to be run on the PYNQ board, though in a future iteration a PYNQ board could be configured as the transmitter.

In [1]:
import sys
import numpy as np
import simpleaudio as sa
import time

# output_command1.py comm1

# Owen Lyke
# Oct 6 2017
# Based on examle code for the simpleaudio library from 

# Takes the desired command as the first commandline argument and plays the corresponding encoded command on the speakers. 

# Define the desired command (from 0 to 31) (the first command can only be 00 or 01, not 10 or 11)
# For example: 	000000 = 0
# 			   	000001 = 1
#			   	000010 = 2
#				000011 = 3 (WOW DUH. I guess what really matters here is that each tone represents two binary digits)

command = int(sys.argv[1])

print("The input command is:")
print(command)
print(" ")


# Convert from the integer command number to the binary shape
binary = [0,0,0,0,0,0]

for i in range(6):
	if(2**(5-i) <= command):
		binary[i] = 1
		command = command - 2**(5-i)

print("The input command is given in binary as:")
print(binary)
print(" ")


# Now make a list of the tone values
tone_arry = [0,0,0]
for i in range(3):
	bit1 = binary[2*(i)] 	# Leading (MSB)
	bit2 = binary[2*(i)+1]	# Trailing (LSB)

	if(bit1):
		if(bit2):
			tone_arry[i] = 3
		else:
			tone_arry[i] = 2
	else:
		if(bit2):
			tone_arry[i] = 1
		else:
			tone_arry[i] = 0

print("The command tone sequence is:")
print(tone_arry)
print(" ")


# Now put that in the full frame:
prefix = [1,3,3,2,0]
suffix = [1,3,3,2]

tone_sequence = prefix+tone_arry+suffix


print("The whole tone sequence is:")
print(tone_sequence)
print(" ")


for i in range(len(tone_sequence)):
	tone_sequence[i] = 2*tone_sequence[i] + 1

print("The whole tone sequence is: [kHz]")
print(tone_sequence)
print(" ")


# Generate the timesteps
sample_rate = 44100
T = 0.186
t = np.linspace(0, T, T * sample_rate, False)

# Generate the sine waves
onekhz = np.sin(1000 * t * 2 * np.pi)
threekhz = np.sin(3000 * t * 2 * np.pi)
fivekhz = np.sin(5000 * t * 2 * np.pi)
sevenkhz = np.sin(7000 * t * 2 * np.pi)

# normalize to 16-bit range
onekhz *= 32767 / np.max(np.abs(onekhz))
threekhz *= 32767 / np.max(np.abs(threekhz))
fivekhz *= 32767 / np.max(np.abs(fivekhz))
sevenkhz *= 32767 / np.max(np.abs(sevenkhz))

# convert to 16-bit data
onekhz = onekhz.astype(np.int16)
threekhz = threekhz.astype(np.int16)
fivekhz = fivekhz.astype(np.int16)
sevenkhz = sevenkhz.astype(np.int16)


# Play necessary audio files
audio = [0]*24*len(onekhz)
start = time.time()
for i in range(len(tone_sequence)):
	if(tone_sequence[i] == 1):
		play_obj = sa.play_buffer(onekhz, 1, 2, sample_rate)
		play_obj.wait_done()
	elif(tone_sequence[i] == 3):
		play_obj = sa.play_buffer(threekhz, 1, 2, sample_rate)
		play_obj.wait_done()
	elif(tone_sequence[i] == 5):
		play_obj = sa.play_buffer(fivekhz, 1, 2, sample_rate)
		play_obj.wait_done()
	else:
		play_obj = sa.play_buffer(sevenkhz, 1, 2, sample_rate)
		play_obj.wait_done()

end = time.time()

print("Total transmission was: [s]")
print(end-start)
		



ModuleNotFoundError: No module named 'simpleaudio'

## Robot Code


In [ ]:
import time
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
from pynq.lib.arduino.arduino_ardumoto import Arduino_Ardumoto

class ArduinoMotorController:
    def __init__(self):
        self.motor = Arduino_Ardumoto(base.ARDUINO)
        self.motor.configure_pins(self.motor.defaultpins)
        self.motor.configure_polarity(self.motor.motorA, self.motor.pol_default)
        self.motor.configure_polarity(self.motor.motorB, self.motor.pol_default)
        self.motor_map = {"left": self.motor.motorA, "right": self.motor.motorB}
        self.dir_map = {"forward": self.motor.forward, "backward": self.motor.backward}
        self.current_dir = "forward"
        self.motor_speeds = {"left": 0,"right": 0}
        self.stop()
        
    def set_direction(self, motor, direction):
        self.motor.set_dir(self.motor_map[motor], self.dir_map[direction])

    def set_speed(self, motor, speed):
        self.motor.stop(self.motor_map[motor])
        new_dir = "forward" if speed >= 0 else "backward"
        if new_dir is not self.current_dir:
            self.set_direction(motor, new_dir)
            self.current_dir = new_dir
        self.motor_speeds[motor] = speed
        speed = sorted((0, abs(speed), 90))[1]
        self.motor.set_speed(self.motor_map[motor], speed)
        if not self.stopped:
            self.motor.run(self.motor_map[motor])
        
    def start(self):
        self.motor.run(self.motor_map["left"])
        self.motor.run(self.motor_map["right"])
        self.stopped = False
        
    def stop(self):
        self.motor.stop(self.motor_map["left"])
        self.motor.stop(self.motor_map["right"])
        self.stopped = True
    
    def turn(self, direction):
        old_left_motor_speed, old_right_motor_speed = self.motor_speeds["left"], self.motor_speeds["right"]
        if direction == "left":
            self.set_speed("left", 0)
            self.set_speed("right", 25)
        else:
            self.set_speed("left", 25)
            self.set_speed("right", 0)
        time.sleep(1.0)
        self.set_speed("left", old_left_motor_speed)
        self.set_speed("right", old_right_motor_speed)       

    def go(self, direction, speed):
        self.start()
        if direction == "forward":
            self.set_speed("left", speed)
            self.set_speed("right", speed)
        else:
            self.set_speed("left", -speed)
            self.set_speed("right", -speed)
        time.sleep(3)
        self.stop()
        
        
arduino_motor_controller = ArduinoMotorController()
arduino_motor_controller.set_speed("right", 0)
arduino_motor_controller.set_speed("left", 0)
arduino_motor_controller.start()

while True:
    print("loop")
    time.sleep(5)
    #continue
    arduino_motor_controller.go("forward", 50)
    arduino_motor_controller.turn("left")
    time.sleep(5)
    arduino_motor_controller.go("reverse", 50)
    arduino_motor_controller.turn("right")